## Demo for Exposing `Neuron Population` and `Nerve`-Related Information from PostgreSQL




In [13]:
from pprint import pprint
import requests

In [14]:
# Initialising the pg database endpoint
pg_endpoint = "http://localhost:8000/competency/"
# pg_endpoint = "https://mapcore-demo.org/devel/flatmap/v4/competency/"

**Before querying neuron population and nerve-related information:**

* We will explore the available queries.
* This will help users locate the relevant query.
* It will also make it easier to track future changes, such as:

  * Changes in query ID
  * Changes in parameters
  * Changes in output

In [25]:
# Get a list of available queries and their labels
response = requests.get(pg_endpoint + 'queries')

if response.status_code in (200, 201):
    pprint(response.json())
else:
    print(f"Error {response.status_code}: {response.json()}")

[{'id': 1, 'label': 'Neuron populations associated with a location'},
 {'id': 2, 'label': 'Neuron populations terminating at a location'},
 {'id': 3, 'label': 'Neuron populations supported by evidence'},
 {'id': 4, 'label': 'Evidence for a neuron population and a species'},
 {'id': 5, 'label': 'Neuron populations connecting two anatomical locations'},
 {'id': 6,
  'label': 'Parasympathetic neuron populations that connect two locations'},
 {'id': 7,
  'label': 'Sympathetic neuron populations that connect two locations'},
 {'id': 8, 'label': 'Motoric neuron populations associated with a location'},
 {'id': 9, 'label': 'Nerves that terminate at a location'},
 {'id': 10,
  'label': 'Nerves projecting (directly or indirectly) to a location'},
 {'id': 11, 'label': 'Nerves associated to a specific neuron population'},
 {'id': 12,
  'label': 'All available nerves and their corresponding neuron populations'},
 {'id': 13,
  'label': 'Nodes targeted by a nerve, including those in the forward '
  

As you can see, queries with IDs 9 to 15 are related to nerve information. The following cell defines functions to:

* Retrieve the complete definition of a query
* Execute a query

The next cells will demonstrate the execution of queries 9 through 15.

In [26]:
# A function to get the query definition
def get_query_parameters(query_id):
    url = pg_endpoint + 'queries/' + str(query_id)
    response = requests.get(url)
    
    if response.status_code in (200, 201):
        return response.json()  # Return the query definition as a JSON object
    else:
        print(f"Error {response.status_code}: {response.json()}")
        return None

# A function to execute a query and return results from PostgreSQL
def execute(query):
    url = pg_endpoint + 'query'
    headers = {"Content-Type": "application/json"}
    
    response = requests.post(url, json=query, headers=headers)
    
    if response.status_code in (200, 201):
        return response.json().get('results', [])  # Safely get 'results' from response
    else:
        print(f"Error {response.status_code}: {response.json()}")
        return None 

### Query 9 -- Nerves that terminate at a location

In [27]:
query_id = 9
query_parameters = get_query_parameters(query_id)

if query_parameters:
    pprint(query_parameters)
else:
    print(f"Failed to retrieve parameters for query {query_id}.")

{'id': 9,
 'label': 'Nerves that terminate at a location',
 'parameters': [{'column': 'feature_id',
                 'label': 'Anatomical terms for locations',
                 'multiple': True,
                 'type': 'string'},
                {'column': 'source_id',
                 'default': 'the latest source is used',
                 'label': 'Knowledge source',
                 'type': 'string'}],
 'results': [{'key': 'source_id',
              'label': 'Knowledge source',
              'type': 'string'},
             {'key': 'nerve_id', 'label': 'Nerve term id', 'type': 'string'},
             {'key': 'label', 'label': 'Nerve term label', 'type': 'string'}]}


**As you can see, there are two parameters required for query 9 (Nerves that terminate at a location):**

1. **`feature_id`** – Anatomical terms for locations (can have multiple values)
2. **`source_id`** – Knowledge source (defaults to the latest source)

The following cell will return a list of nerves that terminate at specified location(s).

The query results include the following fields:

* **`source_id`**: Knowledge source (string)
* **`nerve_id`**: Nerve term ID (string)
* **`label`**: Nerve term label (string)

In [ ]:
query = {
    'query_id': '9', 
    'parameters': [
        {
            'column': 'feature_id', 
            'value': [
                'UBERON:0001258'  # neck of urinary bladder
            ]
        }, 
        {
            'column': 'source_id', 
            'value': 'sckan-2024-09-21'
        }
    ]
}

# Execute the query and print results
query_results = execute(query)

if query_results:
    pprint(query_results)
else:
    print("Query execution failed or returned no results.")


{'keys': ['source_id', 'nerve_id', 'label'],
 'values': [['sckan-2024-09-21', 'ILX:0793559', 'bladder nerve']]}


### Query 10 -- Nerves projecting (directly or indirectly) to a location

In [29]:
# As the parameters for query 10 are the same as for query 9, we proceed directly with query execution
query = {
    'query_id': '10', 
    'parameters': [
        {
            'column': 'feature_id', 
            'value': [
                'UBERON:0001258'  # neck of urinary bladder
            ]
        }, 
        {
            'column': 'source_id', 
            'value': 'sckan-2024-09-21'
        }
    ]
}

# Execute the query and print results
query_results = execute(query)

if query_results:
    pprint(query_results)
else:
    print("Query execution failed or returned no results.")



{'keys': ['source_id', 'nerve_id', 'label'],
 'values': [['sckan-2024-09-21',
             'ILX:0785733',
             'gray communicating ramus of second lumbar nerve'],
            ['sckan-2024-09-21',
             'ILX:0785825',
             'gray communicating ramus of first lumbar nerve'],
            ['sckan-2024-09-21',
             'ILX:0793220',
             'white communicating ramus of first lumbar spinal nerve'],
            ['sckan-2024-09-21',
             'ILX:0793221',
             'white communicating ramus of second lumbar spinal nerve'],
            ['sckan-2024-09-21', 'ILX:0793559', 'bladder nerve'],
            ['sckan-2024-09-21', 'UBERON:0018675', 'pelvic splanchnic nerve'],
            ['sckan-2024-09-21', 'UBERON:0018683', 'lumbar splanchnic nerve']]}


### Query 11 -- Nerves associated to a specific neuron population

In [ ]:
query = {
    'query_id': '11', 
    'parameters': [
        {
            'column': 'source_id', 
            'value': 'sckan-2024-09-21'
        },
        {
            'column': 'path_id', 
            'value': 'ilxtr:neuron-type-keast-4'
        }
    ]
}

# Execute the query and print results
query_results = execute(query)

if query_results:
    pprint(query_results)
else:
    print("Query execution failed or returned no results.")

{'keys': ['source_id', 'nerve_id', 'label'],
 'values': [['sckan-2024-09-21',
             'ILX:0739299',
             'gray communicating ramus of sixth lumbar nerve'],
            ['sckan-2024-09-21',
             'ILX:0793228',
             'gray communicating ramus of first sacral nerve'],
            ['sckan-2024-09-21', 'UBERON:0018675', 'pelvic splanchnic nerve'],
            ['sckan-2024-09-21', 'ILX:0793559', 'bladder nerve']]}


### Query 12 -- All available nerves and their corresponding neuron populations

In [ ]:
query = {
    'query_id': '12', 
    'parameters': [
        {
            'column': 'source_id', 
            'value': 'sckan-2024-09-21'
        }
    ],
    'order': [
        'nerve_id'  # Ordering by 'nerve_id'
    ]
}

# Execute the query and print results
query_results = execute(query)

if query_results:
    pprint(query_results)
else:
    print("Query execution failed or returned no results.")

{'keys': ['source_id', 'nerve_id', 'label', 'path_id'],
 'values': [['sckan-2024-09-21',
             'ILX:0738293',
             'Ganglioglomerular nerve',
             'ilxtr:neuron-type-bolew-unbranched-15'],
            ['sckan-2024-09-21',
             'ILX:0738293',
             'Ganglioglomerular nerve',
             'ilxtr:neuron-type-bolew-unbranched-14'],
            ['sckan-2024-09-21',
             'ILX:0738308',
             'External branch of inferior laryngeal nerve',
             'ilxtr:neuron-type-bolew-unbranched-20'],
            ['sckan-2024-09-21',
             'ILX:0738309',
             'Internal branch of inferior laryngeal nerve',
             'ilxtr:neuron-type-bolew-unbranched-21'],
            ['sckan-2024-09-21',
             'ILX:0738312',
             'Aortic arch depressor nerve',
             'ilxtr:neuron-type-aacar-12'],
            ['sckan-2024-09-21',
             'ILX:0738312',
             'Aortic arch depressor nerve',
             'ilxtr:neuron

### Query 13 -- Nodes targeted by a nerve, including those in the forward connections

In [ ]:
query = {
    'query_id': '13', 
    'parameters': [
        {
            'column': 'feature_id', 
            'value': [
                'ILX:0793559'  # bladder nerve
            ]
        }, 
        {
            'column': 'source_id', 
            'value': 'sckan-2024-09-21'
        }
    ]
}

# Execute the query and print results
query_results = execute(query)

if query_results:
    pprint(query_results)
else:
    print("Query execution failed or returned no results.")


{'keys': ['source_id', 'axon_terminal'],
 'values': [['sckan-2024-09-21', '["UBERON:0000483", ["UBERON:0001258"]]'],
            ['sckan-2024-09-21', '["UBERON:0001135", ["UBERON:0001258"]]'],
            ['sckan-2024-09-21', '["UBERON:0002384", ["UBERON:0001258"]]'],
            ['sckan-2024-09-21', '["UBERON:0035965", ["ILX:0793663"]]'],
            ['sckan-2024-09-21', '["UBERON:0035965", ["ILX:0793664"]]']]}


### Query 14 -- Nodes serving as the origin of a nerve, including those in the forward connections

In [ ]:
query = {
    'query_id': '14', 
    'parameters': [
        {
            'column': 'feature_id', 
            'value': [
                'ILX:0793559'  # bladder nerve
            ]
        }, 
        {
            'column': 'source_id', 
            'value': 'sckan-2024-09-21'
        }
    ]
}

# Execute the query and print results
query_results = execute(query)

if query_results:
    pprint(query_results)
else:
    print("Query execution failed or returned no results.")


{'keys': ['source_id', 'soma'],
 'values': [['sckan-2024-09-21', '["ILX:0738432", []]'],
            ['sckan-2024-09-21', '["ILX:0739295", []]'],
            ['sckan-2024-09-21', '["ILX:0739296", []]'],
            ['sckan-2024-09-21', '["ILX:0739297", []]'],
            ['sckan-2024-09-21', '["ILX:0786933", []]'],
            ['sckan-2024-09-21', '["ILX:0787009", []]'],
            ['sckan-2024-09-21', '["ILX:0788315", []]'],
            ['sckan-2024-09-21', '["ILX:0789862", []]'],
            ['sckan-2024-09-21', '["ILX:0790472", []]'],
            ['sckan-2024-09-21', '["ILX:0793360", []]'],
            ['sckan-2024-09-21', '["UBERON:0002856", []]'],
            ['sckan-2024-09-21', '["UBERON:0002857", []]'],
            ['sckan-2024-09-21', '["UBERON:0002860", []]'],
            ['sckan-2024-09-21', '["UBERON:0005453", []]'],
            ['sckan-2024-09-21', '["UBERON:0006448", []]'],
            ['sckan-2024-09-21', '["UBERON:0006450", []]'],
            ['sckan-2024-09-21', '["UB

### Query 15 -- Nodes serving as the origin or destination of a nerve (considering paths only, not considering forward connection)

In [ ]:
query = {
    'query_id': '15', 
    'parameters': [
        {
            'column': 'feature_id', 
            'value': [
                'ILX:0793559'  # bladder nerve
            ]
        }, 
        {
            'column': 'source_id', 
            'value': 'sckan-2024-09-21'
        }
    ]
}

# Execute the query and print results
query_results = execute(query)

if query_results:
    pprint(query_results)
else:
    print("Query execution failed or returned no results.")


{'keys': ['source_id', 'node_id', 'type'],
 'values': [['sckan-2024-09-21', '["ILX:0738432", []]', 'axon-terminal'],
            ['sckan-2024-09-21', '["ILX:0739295", []]', 'soma'],
            ['sckan-2024-09-21', '["ILX:0739296", []]', 'soma'],
            ['sckan-2024-09-21', '["ILX:0739297", []]', 'soma'],
            ['sckan-2024-09-21', '["ILX:0786933", []]', 'soma'],
            ['sckan-2024-09-21', '["ILX:0787009", []]', 'soma'],
            ['sckan-2024-09-21', '["ILX:0788315", []]', 'soma'],
            ['sckan-2024-09-21', '["ILX:0789862", []]', 'soma'],
            ['sckan-2024-09-21', '["ILX:0790472", []]', 'soma'],
            ['sckan-2024-09-21', '["ILX:0793360", []]', 'soma'],
            ['sckan-2024-09-21',
             '["UBERON:0000483", ["UBERON:0001258"]]',
             'axon-terminal'],
            ['sckan-2024-09-21',
             '["UBERON:0001135", ["UBERON:0001258"]]',
             'axon-terminal'],
            ['sckan-2024-09-21',
             '["UBERON:0002